In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import requests
from bs4 import BeautifulSoup
from datetime import date
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from transformers import TextClassificationPipeline
from transformers import BertTokenizerFast
from transformers import TFBertForSequenceClassification

import warnings
warnings.filterwarnings('ignore')

In [2]:
# LSTM 토크나이저
with open('./src/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)  
# LSTM 모델
model = load_model('./src/model.h5')
# BERT 토크나이저, 모델
loaded_tokenizer = BertTokenizerFast.from_pretrained('./src/bert', from_pt=True)
loaded_model = TFBertForSequenceClassification.from_pretrained('./src/bert', from_pt=True)
classifier = TextClassificationPipeline(tokenizer=loaded_tokenizer, model=loaded_model,
                                            framework='tf', return_all_scores=True) 

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [53]:
df = pd.read_csv('./src/comments_kakao_year.csv')
df

,날짜,댓글
0,2022-05-26,외국인 기관 프로그램 매수다
1,2022-05-26,이게 이제야나오네..
2,2022-05-26,자회사 관계사 또 분할상장해서 5만원까지...
3,2022-05-26,카카오에서
4,2022-05-26,앵두형님이 바닥이란다
...,...,...
147076,2021-04-01,시장에 소외된 저평가주식 하소연 해봐요
147077,2021-04-01,우리나라 실전투자대회 최다우승자입니다
147078,2021-04-01,주주분들 축하합니다
147079,2021-04-01,나스닥 폭등중


In [55]:
df = pd.read_csv('./src/comments_naver_year.csv')
df

,날짜,댓글
0,2022-05-26,30 평단 언제 올까
1,2022-05-26,최고가에 못팔아 먹더니?
2,2022-05-26,내평단
3,2022-05-26,26년차)벌써 한장 벌고 있데이~♡
4,2022-05-26,잠시나마.
...,...,...
37799,2021-04-01,"한투연 입니다.(공매도 제도개선, 금융적..."
37800,2021-04-01,그냥 둘다 사
37801,2021-04-01,여러분 기억 앞에 겸손합시다 !
37802,2021-04-01,문간첩이 주주의결권 폐지한답니다
